In [1]:
import datalabframework as dlf

In [2]:
dlf.project.profile()
dlf.utils.pretty_print(dlf.params.metadata())

engines:
  spark:
    config:
      master: local[*]
      packages:
      - com.jslsolucoes:ojdbc6:11.2.0.1.0
    context: spark
loggers:
  stream:
    enable: true
    severity: info
profile: default
providers:
  ingest:
    format: parquet
    hostname: hdfs-nn
    path: /data/ingest
    service: hdfs
    write:
      coalesce: 2
      options:
        mode: append
        partitionBy:
        - date
  source:
    database: EE.oracle.docker
    hostname: 172.19.0.3
    password: 1234
    port: 1521
    read:
      cache: true
      repartition: 4
    service: oracle
    username: vnpay
resources:
  .inaccount:
    path: account
    provider: source
  .incustomer:
    path: customer
    provider: source
  .outaccount:
    path: account
    provider: ingest
    write:
      option:
        mode: append
        partitionBy:
        - date
  .outcustomer:
    path: customer
    provider: ingest
    write:
      option:
        mode: append
        partitionBy:
        - date
variables:


In [5]:
engine = dlf.engines.get('spark')
spark = engine.context()

In [6]:
# empDF = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:oracle:thin:vnpay/1234@//172.20.0.3:1521/EE.oracle.docker") \
#     .option("dbtable", "customer") \
#     .option("driver", "oracle.jdbc.driver.OracleDriver") \
#     .load()
df_src = engine.read('incustomer')
# df_src.show(5)
# read by resource path and provider
# df_src = engine.read(path='customer', provider='source')

2018-10-17 02:26:26,872 - INFO - 0 -  - jovyan - Oracle_main.ipynb - engine.read - {'format': 'parquet', 'service': 'oracle', 'path': 'customer', 'url': 'jdbc:oracle:thin:vnpay/1234@//172.20.0.3:1521/EE.oracle.docker'}


In [7]:
df_src.show(5)

+-------+-----------------+---------+------------+-----------+-----------+-----+
|CUST_ID|          ADDRESS|     CITY|CUST_TYPE_CD|     FED_ID|POSTAL_CODE|STATE|
+-------+-----------------+---------+------------+-----------+-----------+-----+
|      4|   12 Buchanan Ln|  Waltham|           I|444-44-4444|      02451|   MA|
|     12|      789 Main St|    Salem|           B| 04-3333333|      03079|   NH|
|      1|47 Mockingbird Ln|Lynnfield|           I|111-11-1111|      01940|   MA|
|      5|     2341 Main St|    Salem|           I|555-55-5555|      03079|   NH|
|      8|   472 Freedom Rd|    Salem|           I|888-88-8888|      03079|   NH|
+-------+-----------------+---------+------------+-----------+-----------+-----+
only showing top 5 rows



In [18]:
df_src.rdd.getNumPartitions()

4

In [19]:
# write by resource alias 
engine.write(df_src, 'outcustomer', mode='overwrite')

2018-10-17 02:33:29,877 - INFO - 0 -  - jovyan - Oracle_main.ipynb - engine.write - {'format': 'parquet', 'service': 'hdfs', 'path': 'customer', 'url': 'hdfs://hdfs-nn:8020/data/ingest/customer'}
